# Ciencia de datos
## Práctica 2. Lectura y manipulación de datos con pandas
### Alberto Benavides

Una vez extraídos los datos de los archivos PDF como se mencionó en la [práctica anterior](https://nbviewer.jupyter.org/github/jbenavidesv87/CienciaDatos/blob/master/Notebooks/P1.ipynb), se lidió con los errores de lectura de las librerías y programas con los que se realizó el proceso. Los datos extraídos se guardaron en formato CSV y se ven como

```
2006|01|TOTAL| 'T�tanos\rCIE-10a REV.\rA34| '-'
2006|2006_sem23|TOTAL| 'Hepatitis\raguda tipo A\rCIE-10a REV.\rB15'| '276'
2006|2006_sem30|TOTAL| 'Varicela\rCIE-10a REV.\rB01'| '3 320'
```

Se optó por separar las columnas de los datos con `|` en lugar de `,`, separador por defecto, puesto que en algunos casos la separación por miles se denota con `,` o se utiliza este mismo símbolo para separar nombres de enfermedades. Por otro lado, la segunda columna que corresponde al número de la semana podía contener prefijos del nombre del archivo, como en `2006_sem23` asociado a la semana 23 del año 2006. Aunado a esto, la cuarta columna contiene tanto el nombre de la enfermedad como la CIE correspondiente y suele contener la revisión de la CIE con base en la cual se hizo la clasificación para cada enfermedad. Esto tiende a aparecer como `CIE-10a REV.`. También es posible encontrar símbolos para saltar líneas, espacios o tabulaciones como `\r`, `\n`, etc. Finalmente, es común que el algoritmo de extracción de datos desde tablas en archivos PDF incluya espacios en blanco al inicio o al final de los datos extraídos además de `'` o `"` cuando se toma la información de la celda como textual y no numérica.

Para solucionar todas estas eventualidades, primero se leyeron los archivos generados con [`csv.reader`](https://docs.python.org/3/library/csv.html) especificando que `|` es el separador de columnas con

```python
with open('test/{}enf.csv'.format(year), "r", newline='') as f:
    reader = csv.reader(f, delimiter='|')
```

La información obtenida se almacena en `reader` y, para poder manipular las columnas, se convierten a arreglos con 

```python
lines = list(reader)
```

De esta manera, se puede iterar entre registros

```python
for i in range(len(lines)):
```

tal que, por ejemplo, `lines[i][1]` corresponde al registro `i`, segunda columna (los índices de columnas empiezan en `0`).

Ahora se sobreescriben los datos indeseables con los de interés. Primero se extrajo la semana con

```python
lines[i][1] = lines[i][1][-2:]
```

El índice `[-2:]` se utiliza para tomar desde el penúltimo caracter hasta el último de la columna de la semana. Con ello se elimina todo lo correspondiente a los nombres de archivo y se conserva sólo el número de la semana, desde la `01` hasta la `53`. Por su parte, es necesario forzar la conversión de casos reportados (`lines[i][4]`) a cadenas de texto para poderlas manipular, lo que se logra con

```python
lines[i][4] = str(lines[i][4])
```

En tanto los casos son cadenas de texto, es posible eliminar las `'` con

```python
lines[i][4] = lines[i][4].replace("'", "")
```

que utiliza [`string.replace`](https://docs.python.org/3.3/library/stdtypes.html#str.replace) para sustituir el primer argumento de la función por el segundo en la cadena de texto desde la que se llama el método. Ya que se eliminan las `'` se requiere eliminar espacios iniciales en la cadena mediante

```python
lines[i][4] = re.sub("^\s+", "", lines[i][4])
```

Después, es necesario convertir los `-` a ceros

```python
if lines[i][4][0] is "-":
  lines[i][4] = "0"
```

y los `n.e` o `n.d` a `NA`

```python
lines[i][4] = lines[i][4].replace("n.e", "NA")
lines[i][4] = lines[i][4].replace("n.d", "NA")
```

El trato que se da a la columna de las enfermedades es similar. Primer se eliminan los espacios iniciales y finales

```python
lines[i][3] = re.sub("\s+", " ", lines[i][3])
lines[i][3] = re.sub("^\s+", "", lines[i][3])
```

luego se eliminan los `'`

```python
lines[i][3] = lines[i][3].replace("'", "")
```

se cambian los `\r` por espacios en blanco puesto que los saltos de línea en los cuadros de los PDFs corresponden con espacios en los nombres de las enfermedades

```python
lines[i][3] = lines[i][3].replace("\\r", " ")
```

y, por estandarizar el nombre de las enfermedades, se convierten todas a minúsculas

```python
lines[i][3] = str.lower(lines[i][3]) # a minúsculas
```

por último, cuando se encuentra `cie` en la columna correspondiente al nombre de la enfermedad, se almacenan los caracteres posteriores a `cie-10a rev.` en `cie`, variable temporal que se agregará como columna a la línea con la instrucción [`append`](https://www.tutorialspoint.com/python3/list_append.htm)

```python
if "cie" in lines[i][3]:
  cie = lines[i][3][lines[i][3].find("cie-10a rev.") + 13:] 
  lines[i][3] = lines[i][3][:lines[i][3].find("cie-10a rev.") - 1]
  lines[i].append(cie)
```

Tras corregir todos estos detalles, se procede al guardado de estos archivos en CSV con [`csv.writer`](https://docs.python.org/3/library/csv.html#csv.writer)

```python
with open('csvSemanales/{}enf_limpio.csv'.format(year), "w", newline = "") as f:
  writer = csv.writer(f)
  writer.writerows(lines)
```

Ya que se han guardado los datos limpios, se procede a leerlos en [R](https://www.r-project.org/) con 

```r
enfermedades <- read.csv(file="csvSemanales/enf.csv", header=TRUE, sep=",", stringsAsFactors=FALSE,
colClasses=c("integer", "integer", "character", "character", "integer", "character"))
```

en donde se especifica en el atributo `colClasses` el tipo de dato en que deben leerse las columnas. Enseguida se filtran los registros que en la columna `estado` contienen `TOTAL`, lo cual corresponde a la suma de todos los casos registrados en México durante la semana y el año especificado en el registro. Esto se realiza con [`grep`](https://www.rdocumentation.org/packages/base/versions/3.5.2/topics/grep)

```R
enfermedades <- enfermedades[grep("TOTAL", enfermedades$estado),]
```

Tras hacer el preprocesamiento de datos descrito en la práctica 1, los datos obtenidos se agruparon en una columna con [`cbind`](https://www.rdocumentation.org/packages/base/versions/3.5.2/topics/cbind) para añadirse a un [DataFrame](https://www.rdocumentation.org/packages/base/versions/3.5.2/topics/data.frame) con [`rbind`](https://www.rdocumentation.org/packages/spectacles/versions/0.5-0/topics/rbind)

```R
enfLimpias <- rbind(enfLimpias, cbind(coef(modeloLineal)[2], topFourier[1,1], topFourier[2,1], topFourier[3,1], topFourier[4,1], ac$acf[1], ac$acf[2], ac$acf[min(3, 4)], ac$acf[min(4, 8)], ac$acf[min(5, 16)], ac$acf[min(6, 32)], enfermedad$enfermedad, enfermedad$cie))
```

y se agregó el nombre de los encabezados mediante 

```R
colnames(enfLimpias) <- c("m", "f1", "f2", "f3", "f4", "ac1", "ac2", "ac3", "ac4", "ac5", "ac6", "cie", "t")
```

para luego guardarse en un archivo CSV con

```r
write.csv(enfLimpias, file = "semanalesTodas.csv",row.names=FALSE)
```

Estos datos fueron importados a un *dataframe* en Python mediante Pandas con

```python
df = pd.read_csv('semanalesTodas.csv',sep=',')
```

Más adelante, [StandartScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) se usa para estandarizar las características seleccionadas

```python
features = ["m", "f1", "f2", "f3", "f4", "ac1", "ac2", "ac3", "ac4", "ac5", "ac6"]
x = df.loc[:, features].values
x = StandardScaler().fit_transform(x)
```

y se sobreescriben

```python
df.loc[:, features] = x
```

Por último, para poder utilizar estos datos con la librería de [NumPy](http://www.numpy.org/) es necesario extraer los valores

```python
df = df.values
```

Con esto, los datos están limpios y listos para ser utilizados.